In [1]:
import os
import pandas as pd
import numpy as np
from math import sqrt
import datetime
from datetime import date
from itertools import repeat 
from progressbar import ProgressBar
from sklearn.tree import DecisionTreeClassifier 
from sklearn.linear_model import LinearRegression 
from sklearn import metrics 
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, accuracy_score
import time

In [2]:
start_time = time.time()

In [3]:
#Loading the datasets
os.chdir("../")
os.getcwd()

'C:\\Users\\Asus S\\sakce_git\\process\\Sprint3'

In [34]:
train = pd.read_csv("./data/road-train-pre.csv")
test = pd.read_csv("./data/road-test-pre.csv")

In [43]:
#Sort the datasets and trainsofm to datetime
train['event time:timestamp'] = pd.to_datetime(train['event time:timestamp'])
test['event time:timestamp'] = pd.to_datetime(test['event time:timestamp'])
train=train.sort_values(by=["case concept:name", "event time:timestamp"])
test=test.sort_values(by=["case concept:name", "event time:timestamp"])

# Decision Tree

Train Data

In [6]:
train.to_csv("fixed.csv")

log = dict()
with open('fixed.csv', 'r') as file:
    next(file)
    for line in file:
        line = line.strip()
        if len(line) == 0:
            continue
        parts = line.split(',')
        caseid = parts[2]

        task = parts[3]
        timestamp = parts[5]

        if caseid not in log:
            log[caseid] = [[],[]]

        log[caseid][0].append(task)
        log[caseid][1].append(timestamp)
        
file.close()

os.remove('fixed.csv')

In [7]:
for i in log.keys():
    current = log[i][0]  # recording the current case' events
    
    real_next = current[1:]  # next real event
    real_next.append('New Case')  # adding a 'new case' as real next event for every last event
    
    log[i].append(real_next)  # adding the real next events to the log file

Test Data

In [35]:
test.to_csv("fixed_test.csv")

log_test = dict()
with open('fixed_test.csv', 'r') as file:
    next(file)
    for line in file:
        line = line.strip()
        if len(line) == 0:
            continue
        parts = line.split(',')
        caseid = parts[2]
        
        task = parts[3]
        timestamp = parts[5]

        if caseid not in log_test:
            log_test[caseid] = [[],[]]

        log_test[caseid][0].append(task)
        log_test[caseid][1].append(timestamp)
        
file.close()

os.remove('fixed_test.csv')

In [36]:
for i in log_test.keys():
    current = log_test[i][0]  # current case' events
    
    real_next = current[1:]  # next real event
    real_next.append('New Case')  # adding a 'new case' as real next event for every last event
    log_test[i].append(real_next)

In [37]:
m = 0
for i in log.keys():
    if len(log[i][0]) > m:
        m = len(log[i][0])

In [38]:
delete = []
for i in log_test.keys():
    if len(log_test[i][0]) > m:
        print(i)
        print(log_test[i])
        delete.append(i)
        #m_t = len(log_test[i][0])
        
for i in delete:
    print(i)
    
    test.drop(test.index[test['case concept:name'] == i], inplace = True)

    del log_test[i]

C20817
[['Send Fine', 'Create Fine', 'Insert Fine Notification', 'Add penalty', 'Appeal to Judge', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment'], ['2009-06-29', '2009-06-29', '2009-06-29', '2009-08-28', '2009-09-07', '2010-02-10', '2010-02-24', '2010-03-12', '2010-03-22', '2010-04-16', '2010-05-17', '2010-06-21', '2010-07-19', '2010-08-13', '2010-09-16', '2010-10-30', '2011-01-13', '2011-01-31', '2011-03-16', '2011-09-04'], ['Create Fine', 'Insert Fine Notification', 'Add penalty', 'Appeal to Judge', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'Payment', 'New Case']]
C20817


Store Data Train

In [46]:
train_data = {} 

for i in log.keys():
    for x in log[i][0]:
        case = log[i][0]
        ind = log[i][0].index(x)
        
        if ind not in train_data:
            train_data[ind] = [[],[]]
        
        
        train_data[ind][0].append(case[:ind+1])  # appending the trace
        
        if ind < len(case)-1:
            train_data[ind][1].append(case[ind+1])  # appending the next event of the trace
            
        elif ind == len(case)-1:
            train_data[ind][1].append('New Case')

Store Data Test

In [47]:
test_data = {} 

for i in log_test.keys():
    for x in log_test[i][0]:
        case = log_test[i][0]
        ind = log_test[i][0].index(x)
        
        if ind not in test_data:
            test_data[ind] = [[],[]]
        
        
        test_data[ind][0].append(case[:ind+1])  # appending the trace
        
        if ind < len(case)-1:
            test_data[ind][1].append(case[ind+1])  # appending the next event of the trace
            
        elif ind == len(case)-1:
            test_data[ind][1].append('New Case')

Encoding

In [48]:
cases = list(train['event concept:name'].unique()) + list(test['event concept:name'].unique())
cases.append('New Case')
cases = list(set(cases))
le = preprocessing.LabelEncoder()
le.fit(cases)  # encoding all event names into integers

LabelEncoder()

In [49]:
pbar = ProgressBar()

for i in pbar(train_data.keys()):
    
    encoded = []
    for trace in train_data[i][0]:  # encoding all strings into integers in the trace
        local_encoded = []
        for event in trace:
            local_encoded.append(int(le.transform([event])))
        encoded.append(local_encoded)
    
    train_data[i][0] = np.array(encoded)
    
    
    encoded_next = []
    for g in train_data[i][1]:
        encoded_next.append(int(le.transform([g])))
                            
                            
    train_data[i][1] = np.array(encoded_next)

100% |########################################################################|


In [50]:
pbar = ProgressBar()

for i in pbar(test_data.keys()):
    
    encoded = []
    for trace in test_data[i][0]:  # encoding all strings into integers in the trace
        local_encoded = []
        for event in trace:
            local_encoded.append(int(le.transform([event])))
        encoded.append(local_encoded)
    
    test_data[i][0] = np.array(encoded)
    
    
    encoded_next = []
    for g in test_data[i][1]:
        encoded_next.append(int(le.transform([g])))
                            
                            
    test_data[i][1] = np.array(encoded_next)

100% |########################################################################|


Training

In [51]:
def decision_tree(pos):

    x_train= train_data[pos][0]
    y_train= train_data[pos][1]

    classifier = DecisionTreeClassifier()
    classifier.fit(x_train, y_train)
    
    return classifier

In [52]:
predictors = {}

for i in range(len(test_data)):
    if i >= len(train_data) - 1:
        predictors[i] = decision_tree(len(train_data) - 1)
        
    else:
        predictors[i] = decision_tree(i)

Adding Predictions

In [53]:
pbar = ProgressBar()

for i in pbar(log_test.keys()):
    current = log_test[i][0]
    
    
    encoded = []
    for g in current:
        encoded.append(int(le.transform([g])))
    encoded = np.array(encoded)
    log_test[i].append(encoded)

100% |########################################################################|


In [54]:
pbar = ProgressBar()
for i in pbar(log_test.keys()):
    
    current_encoded = log_test[i][3]
    predictions = []
    
    for x in current_encoded:
        ind = list(current_encoded).index(x)
        
        if ind >= len(train_data) - 1:
            
            tree = predictors[len(train_data) - 1]
            p = current_encoded[:(len(train_data))]
            p = p.reshape(1, -1)
            pred = tree.predict(p)
            pred_string = le.inverse_transform(pred)[0]
            predictions.append(pred_string)
            
            
            
        else:
        
            tree = predictors[ind]
            p = current_encoded[:ind+1]
            p = p.reshape(1, -1)
            pred = tree.predict(p)
            pred_string = le.inverse_transform(pred)[0]
            predictions.append(pred_string)
        
    log_test[i].append(predictions)

100% |########################################################################|


Evaluation

In [55]:
case_names = []
event_names = []
timestamp = []
p_event = []
current_real = []

for i in log_test.keys():
    for x in range(len(log_test[i][0])):
        case_names.append(i)
        event_names.append(log_test[i][0][x])
        timestamp.append(log_test[i][1][x])
        p_event.append(log_test[i][4][x])
        current_real.append(log_test[i][2][x])


frame_dict = {'Case_ID': case_names, 'Event_Name': event_names,
              'TimeStamp': timestamp, 'Next_Event': current_real, 'Predicted_Event': p_event}
predicted_df = pd.DataFrame.from_dict(frame_dict)

event_real = np.array(predicted_df['Next_Event'])
event_pred = np.array(predicted_df['Predicted_Event'])

acc = accuracy_score(event_real, event_pred)
print('Accuracy for event prediction TEST SET: {}%'.format(round(acc, 2) * 100))

Accuracy for event prediction TEST SET: 69.0%


In [56]:
predicted_df

,Case_ID,Event_Name,TimeStamp,Next_Event,Predicted_Event
0,S137581,Create Fine,2009-06-15,Send Fine,Send Fine
1,S137581,Send Fine,2009-11-28,New Case,Insert Fine Notification
2,A42604,Create Fine,2009-06-15,Payment,Send Fine
3,A42604,Payment,2009-06-18,New Case,New Case
4,S136541,Payment,2009-06-15,Create Fine,Create Fine
5,S136541,Create Fine,2009-11-06,New Case,New Case
6,S137809,Payment,2009-06-15,Create Fine,Create Fine
7,S137809,Create Fine,2009-10-06,New Case,New Case
8,S137580,Create Fine,2009-06-15,Payment,Send Fine
9,S137580,Payment,2009-06-19,New Case,New Case


# Linear Regression 

Train Data Preprocessing

In [57]:
#Add new useful columns for the model train
train['position_event']=train.groupby('case concept:name').cumcount()
train['position_event']=train['position_event']+1
train['week_day']=train['event time:timestamp'].dt.dayofweek

In [58]:
train

,eventID,case concept:name,event concept:name,event lifecycle:transition,event time:timestamp,position_event,week_day
295268,1,A1,Send Fine,complete,2006-05-12,1,4
303702,0,A1,Create Fine,complete,2006-07-24,2,0
275275,4294967296,A100,Create Fine,complete,2006-02-08,1,2
321904,4294967297,A100,Send Fine,complete,2006-12-12,2,1
328419,4294967298,A100,Insert Fine Notification,complete,2007-01-15,3,0
339916,4294967299,A100,Add penalty,complete,2007-03-16,4,4
421078,4294967300,A100,Send for Credit Collection,complete,2009-03-30,5,0
328081,8589934595,A10000,Add penalty,complete,2007-01-10,1,2
330638,8589934594,A10000,Insert Fine Notification,complete,2007-02-08,2,3
351792,8589934593,A10000,Send Fine,complete,2007-07-17,3,1


In [59]:
#Encoding all event names into integers
cases = train['event concept:name'].unique().tolist()
cases.insert(0, 'New Case')
le_case = preprocessing.LabelEncoder()
le_case.fit(cases)

LabelEncoder()

In [60]:
#Encoding lifecycle into integers
life = train['event lifecycle:transition'].unique().tolist()
le_life = preprocessing.LabelEncoder()
le_life.fit(life)

LabelEncoder()

In [61]:
#Preprocess data for model train
#Event poistion
x_train_position = np.array(train['position_event']).reshape(-1,1)[:]
#Previous event
x_train_prev = list(train['event concept:name'])
x_train_prev= le_case.transform(x_train_prev)
x_train_prev = np.array(x_train_prev).reshape(-1,1)[:]
# Event
x_train_event = list(train['event concept:name'])
x_train_event.insert(len(train), 'New Case')
x_train_event= le_case.transform(x_train_event)
x_train_event = np.array(x_train_event).reshape(-1,1)[1:]
#Day of the week previous event event
x_train_week = list(train['week_day'])
x_train_week = np.array(x_train_week).reshape(-1,1)[:]
#Timestamp event
train[['event time:timestamp']] = train[['event time:timestamp']].astype(str)
x_train_date = list(train['event time:timestamp'])
x_train_date.insert(len(train), None)
x_train_date=np.array(x_train_date).reshape(-1,1)[1:]
#Timestamp previous event
x_train_date_prev = list(train['event time:timestamp'])
x_train_date_prev=np.array(x_train_date_prev).reshape(-1,1)[:]
#Event Lifecycle
x_train_life = list(train['event lifecycle:transition'])
x_train_life= le_life.transform(x_train_life)
x_train_life = np.array(x_train_life).reshape(-1,1)[:]

In [62]:
#Length case for train set
cases = train.groupby(['case concept:name'])
per_case = pd.DataFrame({'no of events':cases['eventID '].count()})
lst_per_case = per_case["no of events"].tolist()
case_length = []
for length in lst_per_case:
    case_length.extend(repeat(length, length))
x_train_length_case=np.array(case_length).reshape(-1,1)[:]

In [63]:
#Combine features for the model train
x_train_new = np.concatenate((x_train_position,x_train_prev, x_train_event, x_train_week, x_train_date, x_train_date_prev, x_train_length_case, x_train_life), axis=1)

In [64]:
#Add features to new dataframe train
df_train = pd.DataFrame(data=x_train_new, columns=['position_event', 'prev_event', 'event', 'week_day_prev', 'date', 'date_prev', 'case_length', 'lifecycle'])
df_train.loc[df_train['position_event'] == df_train['case_length'], 'event'] = 5
df_train[['date','date_prev']] = df_train[['date','date_prev']].apply(pd.to_datetime)
df_train.loc[df_train['event'] == 5, 'date'] = None
df_train['in_between'] = (df_train['date'] - df_train['date_prev']).dt.days
df_train.loc[df_train['event'] == 5, 'in_between'] = 0
df_train

,position_event,prev_event,event,week_day_prev,date,date_prev,case_length,lifecycle,in_between
0,1,10,2,4,2006-07-24,2006-05-12,2,0,73.0
1,2,2,5,0,NaT,2006-07-24,2,0,0.0
2,1,2,10,2,2006-12-12,2006-02-08,5,0,307.0
3,2,10,4,1,2007-01-15,2006-12-12,5,0,34.0
4,3,4,0,0,2007-03-16,2007-01-15,5,0,60.0
5,4,0,11,4,2009-03-30,2007-03-16,5,0,745.0
6,5,11,5,0,NaT,2009-03-30,5,0,0.0
7,1,0,4,2,2007-02-08,2007-01-10,5,0,29.0
8,2,4,10,3,2007-07-17,2007-02-08,5,0,159.0
9,3,10,2,1,2007-09-03,2007-07-17,5,0,48.0


Train Dummies

In [65]:
#Implementing dummies train
df_train=pd.get_dummies(df_train, columns=['event', 'prev_event', 'week_day_prev', 'position_event', 'lifecycle'])
df_train = df_train.drop(['date', 'date_prev'], 1)
df_train

,case_length,in_between,event_0,event_1,event_2,event_3,event_4,event_5,event_6,event_7,...,position_event_9,position_event_10,position_event_11,position_event_12,position_event_13,position_event_14,position_event_15,position_event_16,position_event_17,lifecycle_0
0,2,73.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,0.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,5,307.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,5,34.0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,60.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,5,745.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,5,0.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
7,5,29.0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,5,159.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,5,48.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Test Data Preprocessing

In [66]:
#Add new useful columns for the model test
test['position_event']=test.groupby('case concept:name').cumcount()
test['position_event']=test['position_event']+1
test['week_day']=test['event time:timestamp'].dt.dayofweek

In [67]:
predicted_events=predicted_df['Predicted_Event'][:].tolist()
test['pred_event']=predicted_events

In [68]:
#Preprocess data for model test
#Event poistion
x_test_position = np.array(test['position_event']).reshape(-1,1)[:]
#Previous event
x_test_prev = test['event concept:name'].tolist()
x_test_prev = le_case.transform(x_test_prev)
x_test_prev = np.array(x_test_prev).reshape(-1,1)[:]
#Predicted Event
x_test_event = test['pred_event'].tolist()
x_test_event= le_case.transform(x_test_event)
x_test_event = np.array(x_test_event).reshape(-1,1)[:]
#Day of the week previous event
x_test_week = test['week_day'].tolist()
x_test_week = np.array(x_test_week).reshape(-1,1)[:]
#Timestamp event
test[['event time:timestamp']] = test[['event time:timestamp']].astype(str)
x_test_date = list(test['event time:timestamp'])
x_test_date.insert(len(test), None)
x_test_date=np.array(x_test_date).reshape(-1,1)[1:]
#Timestamp previous event
x_test_date_prev = list(test['event time:timestamp'])
x_test_date_prev=np.array(x_test_date_prev).reshape(-1,1)[:]
#Event Lifecycle
x_test_life = test['event lifecycle:transition'].tolist()
x_test_life= le_life.transform(x_test_life)
x_test_life = np.array(x_test_life).reshape(-1,1)[:]

In [69]:
#Length case for test set
test_cases = test.groupby(['case concept:name'])
per_case_test = pd.DataFrame({'no of events':test_cases['eventID '].count()})
lst_per_case_test = per_case_test["no of events"].tolist()
case_length_test = []
for length in lst_per_case_test:
    case_length_test.extend(repeat(length, length))
x_test_length_case=np.array(case_length_test).reshape(-1,1)[:]

In [70]:
#Combine features for the model test
x_test_new = np.concatenate((x_test_position ,x_test_prev, x_test_event, x_test_week, x_test_date, x_test_date_prev, x_test_length_case, x_test_life), axis=1)

In [71]:
#Add features to new dataframe test
df_test = pd.DataFrame(data=x_test_new, columns=['position_event', 'prev_event', 'event', 'week_day_prev', 'date', 'date_prev', 'case_length', 'lifecycle'])
df_test.loc[df_test['position_event'] == df_test['case_length'], 'date'] = None
df_test[['date','date_prev']] = df_test[['date','date_prev']].apply(pd.to_datetime)
df_test['in_between'] = (df_test['date'] - df_test['date_prev']).dt.days
df_test.loc[df_test['position_event'] == df_test['case_length'], 'in_between'] = 0
df_test

,position_event,prev_event,event,week_day_prev,date,date_prev,case_length,lifecycle,in_between
0,1,2,10,4,2010-01-19,2009-09-25,5,0,116.0
1,2,10,4,1,2010-08-02,2010-01-19,5,0,195.0
2,3,4,10,0,2010-09-04,2010-08-02,5,0,33.0
3,4,0,5,5,2012-03-26,2010-09-04,5,0,569.0
4,5,11,2,0,NaT,2012-03-26,5,0,0.0
5,1,2,5,4,2010-01-19,2009-09-25,2,0,116.0
6,2,10,2,1,NaT,2010-01-19,2,0,0.0
7,1,10,5,1,2009-08-05,2009-07-28,2,0,8.0
8,2,2,10,2,NaT,2009-08-05,2,0,0.0
9,1,10,5,1,2009-08-17,2009-07-28,6,0,20.0


In [72]:
#Remove cases with more events than the cases in the train set
df_test=df_test[df_test['case_length']<=max(df_train['case_length'])]
df_test

,position_event,prev_event,event,week_day_prev,date,date_prev,case_length,lifecycle,in_between
0,1,2,10,4,2010-01-19,2009-09-25,5,0,116.0
1,2,10,4,1,2010-08-02,2010-01-19,5,0,195.0
2,3,4,10,0,2010-09-04,2010-08-02,5,0,33.0
3,4,0,5,5,2012-03-26,2010-09-04,5,0,569.0
4,5,11,2,0,NaT,2012-03-26,5,0,0.0
5,1,2,5,4,2010-01-19,2009-09-25,2,0,116.0
6,2,10,2,1,NaT,2010-01-19,2,0,0.0
7,1,10,5,1,2009-08-05,2009-07-28,2,0,8.0
8,2,2,10,2,NaT,2009-08-05,2,0,0.0
9,1,10,5,1,2009-08-17,2009-07-28,6,0,20.0


Test Dummies

In [73]:
#Implementing dummies test
df_test=pd.get_dummies(df_test, columns=['event', 'prev_event', 'week_day_prev', 'position_event', 'lifecycle'])
df_test = df_test.drop(['date', 'date_prev'], 1)
df_test

,case_length,in_between,event_0,event_1,event_2,event_3,event_4,event_5,event_6,event_7,...,position_event_7,position_event_8,position_event_9,position_event_10,position_event_11,position_event_12,position_event_13,position_event_14,position_event_15,lifecycle_0
0,5,116.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,5,195.0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,5,33.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,5,569.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,0.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,2,116.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
6,2,0.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,2,8.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,2,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,6,20.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


Feature selection and model training

In [74]:
col_train=df_train.columns
col_test=df_test.columns
features=set(col_train).intersection(col_test)
features.discard('in_between')
X_train = df_train[features] # Features
y_train = df_train['in_between'] # Target variable
X_test = df_test[features] # Features
y_test = df_test['in_between'] # Target variable

#Training the algorithm
regressor = LinearRegression()  
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [75]:
print(regressor.intercept_)
print(regressor.coef_)

-9219895056.45772
[ 3.55027538e+10 -1.63715768e+10 -1.63715768e+10 -9.91379130e+09
  2.50950090e+06 -2.80356440e+01 -1.63715769e+10 -1.63715768e+10
 -9.91379130e+09 -9.91379129e+09 -1.63715768e+10  3.55027539e+10
  3.55027538e+10 -9.91379130e+09  3.55027538e+10  3.55027538e+10
  8.96989357e+00 -1.63715768e+10  3.55027539e+10  3.55027538e+10
 -9.91379130e+09 -1.44614761e+01 -9.91379129e+09  2.35664778e+00
 -7.41829472e+01 -7.02135964e+01 -1.63715766e+10 -1.63715768e+10
 -1.63715768e+10 -2.96169409e+01  3.55027538e+10 -3.51335046e+01
 -9.91379130e+09  3.55027538e+10  3.55027538e+10 -2.19445482e+01
 -1.63715768e+10 -5.41605330e+00 -1.71107564e+01  1.78963991e+01
  1.81056901e+00 -7.97763799e+01 -1.63715768e+10 -2.57717727e+01
 -2.51996405e+01 -1.63715764e+10  3.55027538e+10]


Evaluation

In [76]:
y_pred = regressor.predict(X_test)
df_predictions = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df_predictions

,Actual,Predicted
0,116.0,99.285334
1,195.0,60.174408
2,33.0,88.139399
3,569.0,41.421030
4,0.0,93.444040
5,116.0,10.727663
6,0.0,99.184277
7,8.0,11.066877
8,0.0,110.700993
9,20.0,-10.597334


In [77]:
#R-squared value for the model train
regressor.score(X_train, y_train)

0.7025719220603739

In [78]:
#R-squared value for the model test
regressor.score(X_test, y_test)

-1.3115361077844416

In [79]:
#Root Mean Squared Error of the model
rmse = sqrt(mean_squared_error(y_test, y_pred))
rmse

220.1021770270925

In [80]:
print("--- %s seconds ---" % (time.time() - start_time))

NameError: name 'start_time' is not defined